# <center> Основные типы визуализации </center>

* линейный график;
* гистограмма;
* диаграмма рассеяния (точечная);
* коробчатая диаграмма;
* круговая диаграмма (пироговая/секторная);
* столбчатая диаграмма;
* тепловая карта.



## <center>Предобработка данных</center>

In [1]:
import pandas as pd

covid_data = pd.read_csv('data/covid_data.csv')
display(covid_data.head())

,date,province/state,country,confirmed,deaths,recovered
0,01/22/2020,Anhui,China,1.0,0.0,0.0
1,01/22/2020,Beijing,China,14.0,0.0,0.0
2,01/22/2020,Chongqing,China,6.0,0.0,0.0
3,01/22/2020,Fujian,China,1.0,0.0,0.0
4,01/22/2020,Gansu,China,0.0,0.0,0.0


In [2]:
vaccinations_data = pd.read_csv('data/country_vaccinations.csv')
vaccinations_data = vaccinations_data[
    ['country', 'date', 'total_vaccinations', 
     'people_vaccinated', 'people_vaccinated_per_hundred',
     'people_fully_vaccinated', 'people_fully_vaccinated_per_hundred',
     'daily_vaccinations', 'vaccines']
]
display(vaccinations_data.head())

,country,date,total_vaccinations,people_vaccinated,people_vaccinated_per_hundred,people_fully_vaccinated,people_fully_vaccinated_per_hundred,daily_vaccinations,vaccines
0,Afghanistan,2021-02-22,0.0,0.0,0.0,NaN,NaN,NaN,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
1,Afghanistan,2021-02-23,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
2,Afghanistan,2021-02-24,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
3,Afghanistan,2021-02-25,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
4,Afghanistan,2021-02-26,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."


В таблице `covid_data`:
1. Группируем таблицу по дате и названию страны и рассчитываем суммарные показатели по всем регионам. Тем самым переходим от данных по регионам к данным по странам:

In [3]:
covid_data = covid_data.groupby(
        ['date', 'country'], 
        as_index=False
    )[['confirmed', 'deaths', 'recovered']].sum()

2. Преобразуем даты в формат `datetime` с помощью функции `pd.to_datetime()`:

In [4]:
covid_data['date'] = pd.to_datetime(covid_data['date'])

3. Создадим признак больных на данный момент (`active`). Для этого вычтем из общего числа зафиксированных случаев число смертей и число выздоровевших пациентов:

In [5]:
covid_data['active'] = covid_data['confirmed'] - covid_data['deaths'] - covid_data['recovered']

4. Создадим признак ежедневного прироста числа заболевших, умерших и выздоровевших людей. Для этого отсортируем данные по названиям стран, а затем по датам. После этого произведём группировку по странам и рассчитаем разницу между «вчера и сегодня» с помощью метода `diff()`:

In [6]:
covid_data = covid_data.sort_values(by=['country', 'date'])
covid_data['daily_confirmed'] = covid_data.groupby('country')['confirmed'].diff()
covid_data['daily_deaths'] = covid_data.groupby('country')['deaths'].diff()
covid_data['daily_recovered'] = covid_data.groupby('country')['recovered'].diff()

Итоговый вид таблицы `covid_data`:

In [8]:
display(covid_data.head())

,date,country,confirmed,deaths,recovered,active,daily_confirmed,daily_deaths,daily_recovered
11337,2020-02-24,Afghanistan,1.0,0.0,0.0,1.0,NaN,NaN,NaN
11570,2020-02-25,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0
11807,2020-02-26,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0
12051,2020-02-27,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0
12299,2020-02-28,Afghanistan,1.0,0.0,0.0,1.0,0.0,0.0,0.0


В таблице `vaccinations_data` достаточно будет преобразовать столбцы в формат `datetime`:

In [10]:
vaccinations_data['date'] = pd.to_datetime(vaccinations_data['date'])

display(vaccinations_data.head())

,country,date,total_vaccinations,people_vaccinated,people_vaccinated_per_hundred,people_fully_vaccinated,people_fully_vaccinated_per_hundred,daily_vaccinations,vaccines
0,Afghanistan,2021-02-22,0.0,0.0,0.0,NaN,NaN,NaN,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
1,Afghanistan,2021-02-23,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
2,Afghanistan,2021-02-24,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
3,Afghanistan,2021-02-25,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."
4,Afghanistan,2021-02-26,NaN,NaN,NaN,NaN,NaN,1367.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi..."


In [13]:
# Результат объединения таблиц covid_data и vaccinations_data:

covid_df = pd.read_csv('data/covid_df.csv')

# <center>Графические возможности библиотеки Pandas</center>

Для визуализации *Pandas* использует библиотеку `matplotlib`, о которой мы поговорим в следующем юните. Если попробовать воспользоваться методами визуализации *Pandas* без предварительной установки `matplotlib`, вы получите ошибку `ImportError`. Для установки библиотеки используется стандартная команда, которую нужно выполнить в терминале:

`pip install matplotlib`

## <center>Базовая визуализация в Pandas</center>

Основным методом для создания простейших графиков в *Pandas* является `plot()`.

Основные параметры метода `plot()`:
* `x` — название признака, который будет отложен по оси абсцисс;
* `y` — название признака, который будет отложен по оси ординат;
* `figsize` — размер графика (кортеж из ширины и высоты в дюймах);
* `kind` — тип визуализации. Основные типы:
1) `'line'` — линейный график (по умолчанию);
2) `'bar'` — столбчатая диаграмма;
3) `'barh'` — горизонтальная столбчатая диаграмма;
4) `'hist'` — гистограмма;
5) `'box'` — коробчатая диаграмма (boxplot);
6) `'pie'` — круговая диаграмма;
7) `'scatter'` — диаграмма рассеяния.
* `grid` — отображение сетки (по умолчанию False);
* `legend` — отображение легенды (по умолчанию False);
* `title` — название графика;
* `color` — цвет.